In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install zstandard
!pip install requests

In [ ]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

--2024-11-01 22:14:43--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2024-11-01 22:14:43--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: ‘SQuAD_it-train.json.gz.1’

SQuAD_it-train.json 100%[===================>]   7.37M  --.-KB/s    in 0.07s   

2024-11-01 22:14:44 (106 MB/s) - ‘SQuAD_it-train.json.gz.1’ saved [7725286/7725286]

--

In [ ]:
!gzip -dkv SQuAD_it-*.json.gz

gzip: SQuAD_it-test.json already exists; do you wish to overwrite (y or n)? ^C


In [ ]:
from datasets import load_dataset

#squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")
#quad_it_dataset

In [ ]:
squad_it_dataset["train"][0]

{'title': 'Terremoto del Sichuan del 2008',
 'paragraphs': [{'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
   'qas': [{'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56cdca7862d2951400fa6826',
     'question': 'In quale anno si è verificato il terremoto nel Sichuan?'},
    {'answers': [{'answer_start': 232, 'text': '69.197'}],
     'id': '56cdca7862d2951400fa6828',
     'question': 'Quante persone sono state uccise come risultato?'},
    {'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56d4f9902ccc5a1400d833c0',
     'question': 'Quale anno ha avuto luogo il terremoto del Sichuan?'},
    {'answers': [{'answer_start': 78, 'text': '8.0 Ms e 7.9 Mw'}],
     'id': '56d4f9902ccc5a1400d833c1',
     'question': 'Che cosa ha

In [ ]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [ ]:
data_files = {"train": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
#squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/"
data_files = {
    "train": url + "drugsCom_raw.zip",
    "test": url + "drugsCom_raw.zip",
}
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 215063
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 215063
    })
})

In [ ]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
drug_sample[:3]

NameError: name 'drug_dataset' is not defined

In [ ]:
for split in drug_dataset.keys():
  assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [ ]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)

drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 215063
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 215063
    })
})

In [ ]:
def lowercase_condition(example):
  return {"condition": example["condition"].lower()}

drug_dataset.map(lowercase_condition)

Map:   0%|          | 0/215063 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Filter:   0%|          | 0/215063 [00:00<?, ? examples/s]

Filter:   0%|          | 0/215063 [00:00<?, ? examples/s]

In [ ]:
drug_dataset = drug_dataset.map(lowercase_condition)
# Check that lowercasing worked
drug_dataset

Map:   0%|          | 0/213869 [00:00<?, ? examples/s]

Map:   0%|          | 0/213869 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 213869
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 213869
    })
})

In [ ]:
 def compute_review_length(example):
  return {"review_length": len(example["review"].split())}

In [ ]:
drug_dataset = drug_dataset.map(compute_review_length)
drug_dataset["train"][0]

Map:   0%|          | 0/213869 [00:00<?, ? examples/s]

Map:   0%|          | 0/213869 [00:00<?, ? examples/s]

{'patient_id': 163740,
 'drugName': 'Mirtazapine',
 'condition': 'depression',
 'review': '"I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."',
 'rating': 10.0,
 'date': 'February 28, 2012',
 'usefulCount': 22,
 'review_length': 68}

In [ ]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [159628, 30138, 229124],
 'drugName': ['Insulin inhalation, rapid acting', 'Klonopin', 'Tadalafil'],
 'condition': ['diabetes, type 1', 'bipolar disorde', 'not listed / othe'],
 'review': ['"Horrible"', '"Great."', '"Good."'],
 'rating': [1.0, 10.0, 10.0],
 'date': ['May 13, 2015', 'September 27, 2013', 'November 23, 2011'],
 'usefulCount': [9, 23, 0],
 'review_length': [1, 1, 1]}

In [ ]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

Filter:   0%|          | 0/213869 [00:00<?, ? examples/s]

Filter:   0%|          | 0/213869 [00:00<?, ? examples/s]

{'train': 184622, 'test': 184622}


In [ ]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [ ]:
drug_dataset = drug_dataset.map(lambda x: {'review': html.unescape(x["review"])})

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

In [ ]:
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["review"], truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

CPU times: user 4min, sys: 2.58 s, total: 4min 3s
Wall time: 2min 58s


In [ ]:
%time tokenized_dataset = drug_dataset.map(tokenize_function)

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

CPU times: user 5min 46s, sys: 4.45 s, total: 5min 50s
Wall time: 6min 45s


In [ ]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

def slow_tokenize_function(examples):
  return slow_tokenizer(examples["review"], truncation=True)

tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

In [ ]:
def tokenize_and_split(examples):
  return tokenizer(
      examples["review"],
      truncation=True,
      max_length=128,
      return_overflowing_tokens=True
  )

In [ ]:
result = tokenize_and_split(drug_dataset["train"][3])

In [ ]:
[len(inp) for inp in result["input_ids"]]

[128, 54]

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

ArrowInvalid: Column 8 named input_ids expected length 1000 but got length 1509

In [ ]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split,
    batched=True,
    remove_columns=drug_dataset["train"].column_names
)

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_dataset["train"]), len(drug_dataset["train"])

(275648, 184622)

In [ ]:
def tokenize_and_split(examples):
  result = tokenizer(
      examples["review"],
      truncation=True,
      max_length=128,
      return_overflowing_tokens=True,
  )
  sample_map = result.pop("overflow_to_sample_mapping")
  for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
  return result

In [ ]:
result = tokenize_and_split(drug_dataset["train"][3:5])

In [ ]:
result

{'input_ids': [[101, 107, 16752, 4487, 2707, 14215, 5557, 1115, 1127, 1215, 1111, 6272, 117, 9987, 117, 1105, 11769, 2660, 2386, 172, 5800, 1891, 119, 2563, 3857, 2841, 1113, 1122, 1272, 1122, 1145, 6618, 1654, 1166, 118, 5497, 119, 146, 1138, 1185, 4095, 1115, 1211, 184, 27655, 1110, 2416, 1121, 6656, 120, 1610, 1830, 15658, 117, 1134, 1110, 1198, 1112, 3110, 1112, 1251, 3850, 119, 146, 1138, 1151, 1781, 1122, 1111, 1421, 1552, 117, 1105, 1103, 1363, 2371, 1110, 117, 1122, 3093, 1106, 1301, 1106, 1250, 2411, 119, 146, 1631, 7555, 1196, 146, 1328, 2094, 1208, 119, 146, 1541, 1274, 111, 108, 5347, 1580, 132, 189, 1920, 1106, 3940, 132, 1122, 111, 108, 5347, 1580, 132, 188, 1198, 1106, 5475, 1139, 3472, 119, 1967, 146, 1138, 1178, 1151, 1113, 102], [101, 1122, 170, 1374, 1552, 117, 146, 1274, 111, 108, 5347, 1580, 132, 189, 1221, 1191, 146, 111, 108, 5347, 1580, 132, 1396, 1575, 2841, 113, 146, 1274, 111, 108, 5347, 1580, 132, 189, 1138, 170, 3418, 114, 117, 1133, 1139, 3459, 1202, 1631,

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Map:   0%|          | 0/215063 [00:00<?, ? examples/s]

Map:   0%|          | 0/215063 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 314864
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 314864
    })
})

In [ ]:
drug_dataset.set_format("pandas")

In [ ]:
drug_dataset["train"][:3]

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3


In [ ]:
train_df = drug_dataset["train"][:]
train_df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [ ]:
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "condition": "frequency"})
)
frequencies.head()

,frequency,count
0,Birth Control,38436
1,Depression,12164
2,Pain,8245
3,Anxiety,7812
4,Acne,7435


In [ ]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['frequency', 'count'],
    num_rows: 916
})

In [ ]:
drug_dataset.reset_format()

In [ ]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 172050
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 43013
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 215063
    })
})

In [ ]:
drug_dataset_clean.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/172050 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/43013 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/215063 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("drug-reviews")
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 172050
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 43013
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 215063
    })
})

In [ ]:
for split, dataset in drug_dataset_clean.items():
  dataset.to_csv(f"drug-reviews-{split}.jsonl")

Creating CSV from Arrow format:   0%|          | 0/173 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/44 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/216 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_dataset

data_files = "https://the-eye.eu/public/AI/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
pubmed_dataset

FileNotFoundError: Unable to find 'https://the-eye.eu/public/AI/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst'

In [ ]:
import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 1012.63 MB


In [ ]:
law_dataset_streamed = load_dataset(
    "json",
    data_files="https://the-eye.eu/public/AI/pile_preliminary_components/FreeLaw_Opinions.jsonl.zst",
    split="train",
    streaming=True
)

next(iter(law_dataset_streamed))

FileNotFoundError: Unable to find 'https://the-eye.eu/public/AI/pile_preliminary_components/FreeLaw_Opinions.jsonl.zst'

In [ ]:
base_url = "https://the-eye.eu/public/AI/pile/"
data_files = {
    "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(30)],
    "validation": base_url + "val.jsonl.zst",
    "test": base_url + "test.jsonl.zst",
}
pile_dataset = load_dataset("json", data_files=data_files, streaming=True)
next(iter(pile_dataset["train"]))

FileNotFoundError: Unable to find 'https://the-eye.eu/public/AI/pile/train/00.jsonl.zst'

In [ ]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [ ]:
response.status_code

200

In [ ]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7274',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7274/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7274/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7274/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7274',
  'id': 2629882821,
  'node_id': 'PR_kwDODunzps6ArEt-',
  'number': 7274,
  'title': '[MINOR:TYPO] Fix typo in exception text',
  'user': {'login': 'cakiki',
   'id': 3664563,
   'node_id': 'MDQ6VXNlcjM2NjQ1NjM=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/3664563?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/cakiki',
   'html_url': 'https://github.com/cakiki',
   'followers_url': 'https://api.github.com/users/cakiki/followers',
   'following_url': 'https://api.github.c

In [ ]:
GITHUB_TOKEN = ""
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [ ]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=5_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
  if not issues_path.is_dir():
    issues_path.mkdir(exist_ok=True)


  batch = []
  all_issues = []
  per_page = 100
  num_pages = math.ceil(num_issues / per_page)
  base_url = "https://api.github.com/repos"

  for page in tqdm(range(num_pages)):
    query = f"issues?page={page}&per_page={per_page}&state=all"
    issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
    batch.extend(issues.json())

    if len(batch) > rate_limit and len(all_issues) < num_issues:
      all_issues.extend(batch)
      batch = []
      print(f"Reached Github rate limi, Sleeping for one hour ...")
      time.sleep(60 * 60 + 1)

  all_issues.extend(batch)

  # Replace missing timestamp values with default value
  for issue in all_issues:
    if "created_at" in issue and issue["created_at"] is None:
        issue["created_at"] = "1970-01-01T00:00:00Z"
    if "updated_at" in issue and issue["updated_at"] is None:
        issue["updated_at"] = "1970-01-01T00:00:00Z"

  df = pd.DataFrame.from_records(all_issues)
  df.to_json(f"{issues_path}/{repo}-issues.jsonl",orient="records", lines=True)
  print(f"Downloaded all {len(all_issues)} issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl")

In [ ]:
fetch_issues()

  0%|          | 0/50 [00:00<?, ?it/s]

Downloaded all 5000 issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [ ]:
import pandas as pd
from datasets import Dataset
df = pd.read_json('datasets-issues.jsonl', orient='records', lines=True)
issues_dataset = Dataset.from_pandas(df, split="train")
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason'],
    num_rows: 5000
})

In [ ]:
sample = issues_dataset.shuffle(seed=42).select(range(3))

for url, pr in zip(sample["html_url"], sample["pull_request"]):
  print(f">> URL: {url}")
  print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/issues/5785
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/issues/6151
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/pull/3897
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/3897.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/3897', 'merged_at': '2022-03-14T15:01:08Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/3897.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/3897'}



In [ ]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
respones = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7274',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7274/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7274/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7274/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7274',
  'id': 2629882821,
  'node_id': 'PR_kwDODunzps6ArEt-',
  'number': 7274,
  'title': '[MINOR:TYPO] Fix typo in exception text',
  'user': {'login': 'cakiki',
   'id': 3664563,
   'node_id': 'MDQ6VXNlcjM2NjQ1NjM=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/3664563?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/cakiki',
   'html_url': 'https://github.com/cakiki',
   'followers_url': 'https://api.github.com/users/cakiki/followers',
   'following_url': 'https://api.github.c

In [ ]:
def get_comments(issue_number):
  url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
  response = requests.get(url, headers=headers)
  return [r["body"] for r in response.json()]

get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [ ]:
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

TypeError: string indices must be integers

In [ ]:
issues_with_comments_dataset

NameError: name 'issues_with_comments_dataset' is not defined

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
remote_dataset = load_dataset("lewtun/github-issues", split="train")
remote_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


datasets-issues-with-comments.jsonl:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [ ]:
from datasets import load_dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")
issues_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


datasets-issues-with-comments.jsonl:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [ ]:
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
)

issues_dataset

Filter:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [ ]:
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [ ]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]

In [ ]:
df["comments"][0].tolist()

['Cool, I think we can do both :)',
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).']

In [ ]:
comments_df = df.explode("comments", ignore_index=True)
comments_df.head(4)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...


NameError: name 'comments_dataset' is not defined